<a href="https://colab.research.google.com/github/samyhaff/02456-deep-learning-with-PyTorch/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets elasticsearch faiss_gpu transformers
import datasets
from lfqa_utils import *
from eli5_utils import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
eli5 = datasets.load_from_disk('/content/drive/MyDrive/Deep_Learning_Data/eli5_dataset')
wiki40b_snippets = datasets.load_from_disk('/content/drive/MyDrive/Deep_Learning_Data/wiki40b')

In [9]:
wiki40b_snippets['train'][8991855]

{'_id': '{"datasets_id": 1665419, "wiki_id": "Q179635", "sp": 12, "sc": 653, "ep": 12, "ec": 1223}',
 'datasets_id': 1665419,
 'wiki_id': 'Q179635',
 'start_paragraph': 12,
 'start_character': 653,
 'end_paragraph': 12,
 'end_character': 1223,
 'article_title': 'Heat transfer',
 'section_title': 'Conduction',
 'passage_text': 'from one place to another place without the movement of particles is called conduction, such as when placing a hand on a cold glass of water - heat is conducted from the warm skin to the cold glass, but if the hand is held a few inches from the glass, little conduction would occur since air is a poor conductor of heat. Steady state conduction is an idealized model of conduction that happens when the temperature difference driving the conduction is constant, so that after a time, the spatial distribution of temperatures in the conducting object does not change any'}

In [10]:
eli5['test_eli5'][12345]

{'q_id': '8houtx',
 'title': 'Why does water heated to room temperature feel colder than the air around it?',
 'selftext': '',
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dylcnfk', 'dylcj49'],
  'text': ["Water transfers heat more efficiently than air. When something feels cold it's because heat is being transferred from your skin to whatever you're touching. Since water absorbs the heat more readily than air, it feels colder.",
   "Air isn't as good at transferring heat compared to something like water or steel (sit on a room temperature steel bench vs. a room temperature wooden bench, and the steel one will feel more cold).\n\nWhen you feel cold, what you're feeling is heat being transferred out of you.  If there is no breeze, you feel a certain way.  If there's a breeze, you will get colder faster (because the moving air is pulling the heat away from you), and if you get into water, its quite good at pulling heat from you.   Get out of the water and ha

In [11]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.5.1-linux-x86_64.tar.gz
wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.5.1-linux-x86_64.tar.gz.sha512
shasum -a 512 -c elasticsearch-8.5.1-linux-x86_64.tar.gz.sha512 
tar -xzf elasticsearch-8.5.1-linux-x86_64.tar.gz

elasticsearch-8.5.1-linux-x86_64.tar.gz: OK


--2022-11-20 11:16:00--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.5.1-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580699401 (554M) [application/x-gzip]
Saving to: ‘elasticsearch-8.5.1-linux-x86_64.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  247K 38m14s
    50K .......... .......... .......... .......... ..........  0%  490K 28m46s
   100K .......... .......... .......... .......... ..........  0% 84.1M 19m13s
   150K .......... .......... .......... .......... ..........  0% 78.5M 14m26s
   200K .......... .......... .......... .......... ..........  0%  500K 15m20s
   250K .......... .......... .......... .......... ..........  0% 75.8M 12m48s
   300K .......... .......... .......... .......... ..........  0% 83.2M

In [26]:
import os
from subprocess import Popen, PIPE, STDOUT
# Run Elasticsearch as a background process
!chown -R daemon:daemon elasticsearch-8.5.1
es_server = Popen(args=['elasticsearch-8.5.1/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))
# Wait until Elasticsearch has started
!sleep 30

In [27]:
!curl -X GET "localhost:9200/?pretty"

curl: (7) Failed to connect to localhost port 9200: Connection refused


In [24]:
es_client = Elasticsearch([{'host': 'localhost', 'port': '9200'}])